# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --upgrade --no-deps --force-reinstall ../lib/ait_sdk-0.1.3-py3-none-any.whl

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Processing /workdir/root/lib/ait_sdk-0.1.3-py3-none-any.whl
  Attempting uninstall: ait-sdk
    Found existing installation: ait-sdk 0.1.3
    Uninstalling ait-sdk-0.1.3:
      Successfully uninstalled ait-sdk-0.1.3


In [2]:
from pathlib import Path
import pprint
from ait_sdk.test.hepler import Helper

In [3]:
# settings cell

# mounted dir
root_dir = Path('/workdir/root/ait')

ait_name='dev_template'
ait_version='0.1'

ait_full_name=f'{ait_name}_{ait_version}'
ait_dir = root_dir / ait_full_name

td_name=f'{ait_name}_test'

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
invenotory_root_dir=r'C:\m-akita\github\qai-testbed\exp\788_AIT_Dev_refactor\ait\dev_template_0.1\local_qai\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
is_container = True
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True


In [4]:
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point,
                ait_dir=ait_dir,
                ait_full_name=ait_full_name)

In [5]:
# health check

helper.get_bk('/health-check')
helper.get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [6]:
# deploy AIT
if is_init_ait:
    helper.deploy_ait_non_build()
else:
    print('skip deploy AIT')

<Response [422]>
{'Code': 'T59000',
 'Message': 'database error: (psycopg2.errors.UndefinedColumn) column '
            'M_TestRunner.landing_page does not exist\n'
            'LINE 1: ..."M_TestRunner".quality AS "M_TestRunner_quality", '
            '"M_TestRun...\n'
            '                                                             ^\n'
            '\n'
            '[SQL: SELECT "M_TestRunner".id AS "M_TestRunner_id", '
            '"M_TestRunner".name AS "M_TestRunner_name", '
            '"M_TestRunner".description AS "M_TestRunner_description", '
            '"M_TestRunner".author AS "M_TestRunner_author", '
            '"M_TestRunner".email AS "M_TestRunner_email", '
            '"M_TestRunner".version AS "M_TestRunner_version", '
            '"M_TestRunner".quality AS "M_TestRunner_quality", '
            '"M_TestRunner".landing_page AS "M_TestRunner_landing_page" \n'
            'FROM "M_TestRunner" \n'
            'WHERE "M_TestRunner".name = %(name_1)s AND "M_TestRun

In [7]:
# add inventories

if is_init_inventory:
    inv1_name = helper.post_inventory('iris_data', 'dataset', 'UNIX_FILE_SYSTEM', 
                                      f'{invenotory_root_dir}\\iris_data\\tableconvert_csv_4nryby.csv',
                                      'iris data set', ['csv'])
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [8]:
# get ait_json and inventory_jsons

res_json = helper.get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = helper.get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

inv_1_json = helper.get_inventory(inv1_name)

<Response [200]>
<Response [500]>


KeyError: 'TestRunners'

In [ ]:
# add teast_descriptions

helper.post_td(td_name, 5,
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"1", "RelationalOperatorId":gt_id}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"sepal.length"}
                ]
               })

In [ ]:
# get test_description_jsons
td_1_json = helper.get_td(td_name)

In [ ]:
# run test_descriptions
helper.post_run_and_wait(td_1_json['Id'])

In [ ]:
res_json = helper.get_td_detail(td_1_json['Id'])
pprint.pprint(res_json)

In [ ]:
# generate report
res = helper.post_report(td_1_json['Id'])
pprint.pprint(res)